# Risk-Parity Portfolio Analysis
In this notebook, we will analyze the risk-parity portfolio by downloading financial data, calculating returns, computing risk-parity weights, and evaluating portfolio performance.

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

## Step 1: Download Financial Data
We will download front-month futures data for S&P500, 10-year Treasuries, gold, and US dollar using the `yfinance` library.

In [2]:
# Download front-month futures data
symbols = ['ES=F', 'ZN=F', 'GC=F', 'DX=F']
data = yf.download(symbols)

## Step 2: Resample Data
To reduce noise, we will resample the data to monthly frequency.

In [3]:
# Resample data to monthly frequency
data = data.resample('M').last()
data.index = pd.to_datetime(data.index)

## Step 3: Clean and Prepare Data
We will subset adjusted close prices, fill NaNs with the value known at time `t`, and drop rows with unknown prices.

In [4]:
# Subset adjusted close prices and fill NaNs
prices = data['Adj Close'].ffill().dropna()
prices.index = pd.to_datetime(prices.index)

## Step 4: Calculate Returns
We will calculate the returns of the prices.

In [5]:
# Compute logarithmic returns
returns = prices.pct_change().fillna(0)

## Step 5: Compute Risk-Parity Weights
We will compute the risk-parity weights using a 36-month rolling window.

In [6]:
def compute_risk_parity_weights(returns, window_size=36):
    # Compute volatility known at time t
    rolling_vol = returns.rolling(window_size).std()
    rolling_inverse_vol = 1 / rolling_vol
    # Divide inverse volatility by the sum of inverse volatilities
    risk_parity_weights = rolling_inverse_vol.apply(
        lambda column: column / rolling_inverse_vol.sum(1)
    )
    # Shift weights by one period to use only information available at time t
    return risk_parity_weights.shift(1)

risk_parity_weights = compute_risk_parity_weights(returns, 36)

## Step 6: Calculate Weighted Returns
We will calculate the weighted returns using the risk-parity weights.

In [7]:
# Calculate weighted returns
weighted_returns = (returns * risk_parity_weights).dropna()
risk_parity_portfolio_returns = weighted_returns.sum(axis=1)

## Step 7: Evaluate Portfolio Performance
We will evaluate the performance of the risk-parity portfolio by calculating various metrics such as annualized mean return, annualized volatility, skewness, kurtosis, maximum drawdown, Sharpe ratio, Sortino ratio, and Calmar ratio.

In [8]:
def evaluate_performance(returns, freq=12):
    annualized_mean_return = (returns.mean() * freq)
    print()
    print(f"annualized_mean_return: {np.round(annualized_mean_return * 100, 2)}%")
    annualized_volatility = (returns.std() * np.sqrt(freq))
    print(f"annualized_volatility: {np.round(annualized_volatility * 100, 2)}%")
    skewness = (returns.skew())
    print(f"skewness: {np.round(skewness, 1)}")
    kurtosis = (returns.kurtosis())
    print(f"kurtosis: {np.round(kurtosis)}")
    cum_returns = np.exp(returns.cumsum())
    drawdowns = (cum_returns.cummax() - cum_returns) / cum_returns.cummax()
    max_drawdown = np.round(drawdowns.max(), 2)
    print(f"max_drawdown: {np.round(max_drawdown * 100, 2)}%")
    sharpe_ratio = (annualized_mean_return / annualized_volatility).round(1)
    print(f"sharpe_ratio: {sharpe_ratio}")
    downside_volatility = returns[returns < 0].std() * np.sqrt(freq)
    sortino_ratio = (annualized_mean_return / downside_volatility).round(1)
    print(f"sortino_ratio: {sortino_ratio}")
    calmar_ratio = (annualized_mean_return / max_drawdown).round(1)
    print(f"calmar_ratio: {calmar_ratio}")
    print()
    plt.plot(cum_returns-1, label='Cumulative Returns')
    plt.plot(cum_returns.cummax()-1, label='Cumulative Max', linewidth=.5)
    plt.fill_between(
        drawdowns.index, -drawdowns, color='red', alpha=0.5, label="Drawdowns"
    )
    # Setting x-axis major locator to each year and formatter
    plt.gca().xaxis.set_major_locator(mdates.YearLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    # Setting major formatter for y-axis
    plt.gca().yaxis.set_major_locator(ticker.MultipleLocator(0.1))
    # Adding grid with vertical lines for each year
    plt.grid(True, which='major', linestyle='--', color='grey')
    # Rotate x-axis labels by 45 degrees
    plt.xticks(rotation=45)
    # Adjusting the legend to include all plots
    plt.legend(loc='best')
    # plt.yscale("log")
    plt.title("Cumulative Returns of Risk-Parity Portfolio")
    plt.savefig("risk_parity_returns.png")
    return None

evaluate_performance(risk_parity_portfolio_returns)